Import necassary libraries

In [ ]:
import pandas as pd

Import and display the dataset. find other dataset that has missing value and imbalanced class (target)

In [21]:
df = pd.read_csv('https://github.com/itsliterallymonique/Two-Class-Classification-on-South-African-Heart-Disease-dataset/raw/refs/heads/main/saheart.csv', header=0, index_col=None)
df.head()

,row.names,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
0,1,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
1,2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
2,3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
3,4,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
4,5,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


Check total missing value

METHOD 1: simply drop row of missing value

In [9]:
df.shape
df.isnull().sum()

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [10]:
df_drop_na = df.dropna()

In [11]:
df_drop_na.isnull().sum()

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0


In [12]:
df_drop_na.shape

(463, 11)

do classification

In [34]:
X = pd.get_dummies(df_drop_na.iloc[:, 1:-1])
y = df_drop_na.iloc[:, -1]

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [36]:
from sklearn import tree
model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [37]:
y_pred = model.predict(X_test)

In [38]:
from sklearn.metrics import mean_squared_error, accuracy_score,confusion_matrix

result_acc = accuracy_score(y_test,y_pred)
print(f'Accuracy: {result_acc}')

result_conf = confusion_matrix(y_test,y_pred)
print(f'Confusion Matrix: {result_conf}')

Accuracy: 0.6618705035971223
Confusion Matrix: [[66 23]
 [24 26]]


METHOD 2: data imputation

In [40]:
from sklearn.impute import KNNImputer

# Select only numerical features for imputation

numerical_features = df.select_dtypes(include=['number']).columns
df_numerical = df[numerical_features]

In [41]:
# Create and apply the KNNImputer
imputer = KNNImputer(n_neighbors=5, weights='uniform')
df_impute_numerical = pd.DataFrame(imputer.fit_transform(df_numerical), columns=numerical_features)

In [42]:
# If you need to keep the categorical features, merge them back with the imputed data
df_impute = pd.concat([df_impute_numerical, df.drop(columns=numerical_features)], axis=1)

In [43]:
df_impute.head()

,row.names,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd,famhist
0,1.0,160.0,12.00,5.73,23.11,49.0,25.30,97.20,52.0,1.0,Present
1,2.0,144.0,0.01,4.41,28.61,55.0,28.87,2.06,63.0,1.0,Absent
2,3.0,118.0,0.08,3.48,32.28,52.0,29.14,3.81,46.0,0.0,Present
3,4.0,170.0,7.50,6.41,38.03,51.0,31.99,24.26,58.0,1.0,Present
4,5.0,134.0,13.60,3.50,27.78,60.0,25.99,57.34,49.0,1.0,Present


Do classification

In [44]:
X = pd.get_dummies(df_drop_na.iloc[:, 1:-1])
y = df_drop_na.iloc[:, -1]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [46]:
model = tree.DecisionTreeClassifier()
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [47]:
y_pred = model.predict(X_test)

In [48]:
result_acc = accuracy_score(y_test,y_pred)
print(f'Accuracy: {result_acc}')

result_conf = confusion_matrix(y_test,y_pred)
print(f'Confusion Matrix: {result_conf}')

Accuracy: 0.6546762589928058
Confusion Matrix: [[77 14]
 [34 14]]


Handling imbalanced classes

In [50]:
df['chd'].value_counts()

,count
chd,
0,302
1,160


In [56]:
from imblearn.over_sampling import SMOTE

do classification

In [66]:
oversample = SMOTE()

if y.value_counts().min() > 1:
    oversample = SMOTE()
    X, y = oversample.fit_resample(X, y)
else:
    print("SMOTE cannot be applied due to a class with only 1 sample.")

SMOTE cannot be applied due to a class with only 1 sample.
